In [12]:
import numpy as np
import numba as nb
import matplotlib.pyplot as plt
import timeit
%matplotlib inline

## Мои вопросы.

1. Как понять насколько используется процессор. Мы можем измерить время выполнения. Тоже самое про память.
1. Модель использования вычислительных ресурсов. Это типо как линия регрессии в мат статистике? Есть выходной набор времени выполнения программы, хотим узнать какая зависимость от m_sz, n_sz, k_sz.
1. Как искать размеры кэшей?

## Задание 2

Реализуйте произведение матриц $A\in Mat(N\times K)$, $B\in Mat(K\times M)$ согласно определению
$$
C_{n,m}=\sum_{k=1}^K A_{n,k}B_{k,m}.
$$
Сравните быстродействие реализаций на основе numpy.sum, с помощью numba и стандартный метод numpy.dot.
Насколько полно используется процессор? 
Сколько используется памяти?

In [ ]:
def no_nb_matmul(in_mtrx1: np.ndarray, in_mtrx2: np.ndarray) -> np.ndarray:
    num_rows1, num_cols1 = in_mtrx1.shape
    num_cols2 = in_mtrx2.shape[1]
    # num_rows2, num_cols2 = in_mtrx2.shape
    # assert (num_cols1 == num_rows2, "Incorrect input matrices sizes")
    out_mtrx = np.zeros(shape=(num_rows1, num_cols2), dtype=np.float64)
    for n in range(num_rows1):
        for m in range(num_cols2):
            for k in range(num_cols1):
                out_mtrx[n, m] += in_mtrx1[n, k]*in_mtrx2[k, m]
    return out_mtrx

@nb.njit('float64[:,:](float64[:,:],float64[:,:])', parallel = False) # 2D-arrays
def nb_matmul(in_mtrx1: np.ndarray, in_mtrx2: np.ndarray) -> np.ndarray:
    num_rows1, num_cols1 = in_mtrx1.shape
    num_cols2 = in_mtrx2.shape[1]
    # num_rows2, num_cols2 = in_mtrx2.shape
    # assert (num_cols1 == num_rows2, "Incorrect input matrices sizes")
    out_mtrx = np.zeros(shape=(num_rows1, num_cols2), dtype=np.float64)
    for n in range(num_rows1):
        for m in range(num_cols2):
            for k in range(num_cols1):
                out_mtrx[n, m] += in_mtrx1[n, k]*in_mtrx2[k, m]
    return out_mtrx

def sum_matmul(in_mtrx1: np.ndarray, in_mtrx2: np.ndarray) -> np.ndarray:
    num_rows1, num_cols1 = in_mtrx1.shape
    num_cols2 = in_mtrx2.shape[1]
    out_mtrx = np.empty(shape=(num_rows1, num_cols2), dtype=np.float64)
    for n in range(num_rows1):
        for m in range(num_cols2):
            out_mtrx[n, m] = np.sum(in_mtrx1[n, :]*in_mtrx2[:, m])
    return out_mtrx

In [14]:
a = np.array([[1, 2, 3], [4, 5, 6]])
b = np.array([[1, 2], [3, 4], [5, 6]])
print(np.multiply(a[0,:], b[:,0]))
print(a[0,:]*b[:,0])

[ 1  6 15]
[ 1  6 15]


In [48]:
# generate input data
np.random.seed(0)
num_repetitions = 5

num_rows1 = 10
num_cols2 = 10
num_cols1 = 10000

mtrx1 = np.random.random(size=(num_rows1, num_cols1))
mtrx2 = np.random.random(size=(num_cols1, num_cols2))

In [49]:
# times_no_numba = timeit.repeat(stmt='my_matmul(mtrx1, mtrx2)', globals={"my_matmul": no_nb_matmul, "mtrx1": mtrx1, "mtrx2": mtrx2},
#                             repeat=num_repetitions, number=10)
# print(f"No Numba mean time: {np.mean(times_no_numba)} sec")
times_numba = timeit.repeat(stmt='my_matmul(mtrx1, mtrx2)', globals={"my_matmul": nb_matmul, "mtrx1": mtrx1, "mtrx2": mtrx2},
                            repeat=num_repetitions, number=10)
print(f"Numba mean time: {np.mean(times_numba)} sec")
times_numpy = timeit.repeat(stmt='numpy_matmul(mtrx1, mtrx2)', globals={"numpy_matmul": np.dot, "mtrx1": mtrx1, "mtrx2": mtrx2},
                            repeat=num_repetitions, number=10)
print(f"Numpy mean time: {np.mean(times_numpy)} sec")
times_sum = timeit.repeat(stmt='sum_matmul(mtrx1, mtrx2)', globals={"sum_matmul": sum_matmul, "mtrx1": mtrx1, "mtrx2": mtrx2},
                            repeat=num_repetitions, number=10)
print(f"Sum mean time: {np.mean(times_sum)} sec")

Numba mean time: 0.009821439976803958 sec
Numpy mean time: 0.0016537800431251526 sec
Sum mean time: 0.01757897997740656 sec


## Задание 3

## Задание 4

В простейшем алгоритме произведения матриц используются три цикла: перебирающие элементы $n$ и $m$  матрицы $C$
и суммирующие по $k$. 
Сравните время перемножения матриц в зависимости от порядка циклов.
Оцените объем кэшей разных уровней, проводя эксперименты для матриц разного размера.

In [50]:
@nb.njit('float64[:,:](float64[:,:],float64[:,:])', parallel = False) # 2D-arrays
def nb_matmul_nmk(in_mtrx1: np.ndarray, in_mtrx2: np.ndarray) -> np.ndarray:
    num_rows1, num_cols1 = in_mtrx1.shape
    num_cols2 = in_mtrx2.shape[1]
    # num_rows2, num_cols2 = in_mtrx2.shape
    # assert (num_cols1 == num_rows2, "Incorrect input matrices sizes")
    out_mtrx = np.zeros(shape=(num_rows1, num_cols2), dtype=np.float64)
    for n in range(num_rows1):
        for m in range(num_cols2):
            for k in range(num_cols1):
                out_mtrx[n, m] += in_mtrx1[n, k]*in_mtrx2[k, m]
    return out_mtrx

@nb.njit('float64[:,:](float64[:,:],float64[:,:])', parallel = False) # 2D-arrays
def nb_matmul_nkm(in_mtrx1: np.ndarray, in_mtrx2: np.ndarray) -> np.ndarray:
    num_rows1, num_cols1 = in_mtrx1.shape
    num_cols2 = in_mtrx2.shape[1]
    # num_rows2, num_cols2 = in_mtrx2.shape
    # assert (num_cols1 == num_rows2, "Incorrect input matrices sizes")
    out_mtrx = np.zeros(shape=(num_rows1, num_cols2), dtype=np.float64)
    for n in range(num_rows1):
        for k in range(num_cols1):
            for m in range(num_cols2):
                out_mtrx[n, m] += in_mtrx1[n, k]*in_mtrx2[k, m]
    return out_mtrx

@nb.njit('float64[:,:](float64[:,:],float64[:,:])', parallel = False) # 2D-arrays
def nb_matmul_knm(in_mtrx1: np.ndarray, in_mtrx2: np.ndarray) -> np.ndarray:
    num_rows1, num_cols1 = in_mtrx1.shape
    num_cols2 = in_mtrx2.shape[1]
    # num_rows2, num_cols2 = in_mtrx2.shape
    # assert (num_cols1 == num_rows2, "Incorrect input matrices sizes")
    out_mtrx = np.zeros(shape=(num_rows1, num_cols2), dtype=np.float64)
    for k in range(num_cols1):
        for n in range(num_rows1):
            for m in range(num_cols2):
                out_mtrx[n, m] += in_mtrx1[n, k]*in_mtrx2[k, m]
    return out_mtrx

## Задание 5

In [ ]:
@nb.njit('float64[:,:](float64[:,:],float64[:,:])', parallel = True, nogil = False) # 2D-arrays
def nb_matmul_parallel(in_mtrx1: np.ndarray, in_mtrx2: np.ndarray) -> np.ndarray:
    num_rows1, num_cols1 = in_mtrx1.shape
    num_cols2 = in_mtrx2.shape[1]
    # num_rows2, num_cols2 = in_mtrx2.shape
    # assert (num_cols1 == num_rows2, "Incorrect input matrices sizes")
    out_mtrx = np.zeros(shape=(num_rows1, num_cols2), dtype=np.float64)
    for n in nb.prange(num_rows1):
        for m in range(num_cols2):
            for k in range(num_cols1):
                out_mtrx[n, m] += in_mtrx1[n, k]*in_mtrx2[k, m]
    return out_mtrx

## Задания.

1. Изучите реализацию многомерных массивов numpy.ndarray и работу с ними в numba.
Посмотрите ноутбук [FastPython.](../FastPython.ipynb) 

2. Реализуйте произведение матриц $A\in Mat(N\times K)$, $B\in Mat(K\times M)$ согласно определению
$$
C_{n,m}=\sum_{k=1}^K A_{n,k}B_{k,m}.
$$
Сравните быстродействие реализаций на основе numpy.sum, с помощью numba и стандартный метод numpy.dot.
Насколько полно используется процессор? 
Сколько используется памяти?

3. Составьте модель использования вычислительных ресурсов функцией на основе numba.jit из предыдущего пункта.
Размеры матриц должны быть параметрами модели.
Проведите вычислительные эксперименты, чтобы подобрать параметры модели.
Экстраполируйте результат на большие матрицы, сделайте экспериментальную проверку.

4. В простейшем алгоритме произведения матриц используются три цикла: перебирающие элементы $n$ и $m$  матрицы $C$
и суммирующие по $k$. 
Сравните время перемножения матриц в зависимости от порядка циклов.
Оцените объем кэшей разных уровней, проводя эксперименты для матриц разного размера.

5. Обновите функцию для перемножения матриц, используя несколько потоков с помощью numba.prange.
Обновите модель использования вычислительных ресурсов, принимая во внимание число потоков.
Оцените параметры модели из эксперимента.
Какое [параллельное ускорение](https://en.wikipedia.org/wiki/Amdahl%27s_law) вы получили?

6. Сможете ли вы реализовать реализовать на С более быстрый вариант перемножения матриц, чем на numba?

7. Реализуйте быстрое произведение матриц, например, используйте [алгоритм Штрассена](https://en.wikipedia.org/wiki/Strassen_algorithm).
Оцените, на матрицах какого размера быстрое произведение матриц быстрее, чем стандартная реализация.
Какой из методов дает меньшую погрешность вычислений?


# Дополнительные задания

1. Реализуйте вычисление матрицы дискретизованного конечными разностями оператора Лапласа:
$$
Lf_{n,k}=4f_{n,k}-f_{n+1,k}-f_{n-1,k}-f_{n,k+1}-f_{n,k-1}.
$$

Функция $f$ задана на квадратной решетке своими значениями в узлах $f_{n,k}$, $n=0\ldots N$, $k=0\ldots K$.
Мы будем предполагать периодические граничные условия, в этом случае арифметические операции над индексом $n$ выполняются по модулю $N$, а по индексу $k$ по модулю $K$.
Хотя значения функции и хранятся в двухмерном массиве, с точки зрения матричных вычислений значения функции $f$ в узлах образуют вектор, а оператор $L$ действует на него умножением на матрицу.
Если нас смущает двойной индекс $(n,k)$ у $f$, то мы можем держать в голове, что это просто удобное обозначение для одного числа $n*K+k$, показывающего, в какой ячейке памяти хранится коэффициент вектора.
Оператор $L$ может быть задан своей матрицей:
$$
Lf_{n,k}=\sum_{n',k'}L_{n,k,n',k'} f_{n',k'},
$$
где $(n,k)$ - номер строки матрицы, а $(n',k')$ - номер столбца. 
Двойные индексы можно свернуть снова в обычные числа, тогда матрица будем иметь два индекса (строки и столбец), как нам более привычно. 
Однако, использование двойных индексов позволяет нам записать матрицу в очень простом виде:
$$
L_{n,k,n',k'}=
\begin{cases}
4, & n=n'\text{ и }k=k',\\
-1, & n=n'\pm 1\text{ и }k=k'\text{ или }n=n'\text{ и }k=k'\pm1,\\
0, & \text{в остальных случаях}. 
\end{cases}
$$

2. Матрица $L$ [разреженная](https://ru.wikipedia.org/wiki/%D0%A0%D0%B0%D0%B7%D1%80%D0%B5%D0%B6%D0%B5%D0%BD%D0%BD%D0%B0%D1%8F_%D0%BC%D0%B0%D1%82%D1%80%D0%B8%D1%86%D0%B0), т.е. большинство ее элементов равно нулю. 
Сохраните матрицу $L$ в виде разреженной матрицы из пакета [scipy.sparse](https://docs.scipy.org/doc/scipy/reference/sparse.html).
Какой способ хранения разреженной матрицы подходит лучше всего для матрицы оператора Лапласа?
Какими преимуществами обладает представление матрицы в разреженном виде?

3. Выше приведена формула для вычисления матрицы $L$ на векторах $f$, которая может быть реализована в виде алгоритма более быстрого, чем умножение на произвольню матрицу. Реализуйте функцию, которая будет вычислять произведение $L$ на $f$ без явного использования матрицы $L$. Воспользуйтесь [numpy](https://numpy.org/), постарайтесь реализовать как можно более быстрый код. Сравните полученное быстродействие с максимальной производительностью процессора вашего компьютера, согласно спецификации. 

4. Перепишите функцию из предыдущего задания, используя [numba](https://numba.pydata.org/). Какое ускорение мы можем ожидать, за счет чего оно достигается? Реализуйте наиболее эффективный код, учитывая аппаратные особенности компьютера. Убедитесь, что производительность вашего кода выше, чем умножение на разреженную матрицу, полученную выше. Как близко вы подобрались к пиковой теоретической производительности вашего компьютера?

# Литература

1. Ван Лоун Чарльз Ф., Голуб Джин Х. Матричные вычисления. Глава 1.

1. [NumPy](https://numpy.org/doc/stable/contents.html)

1. [Numba: A High Performance Python Compiler.](https://numba.pydata.org/) [Performance Tips](https://numba.pydata.org/numba-doc/latest/user/performance-tips.html)

1. [JAX: Autograd and XLA](https://github.com/google/jax)

1. [xeus-cling: a Jupyter kernel for C++](https://github.com/jupyter-xeus/xeus-cling)

1. [Minimal C kernel for Jupyter](https://github.com/brendan-rius/jupyter-c-kernel)

1. Micha Gorelick, Ian Ozsvald.
[High Performance Python](https://www.oreilly.com/library/view/high-performance-python/9781449361747/) 

1. [Performance Tips of NumPy ndarray](https://shihchinw.github.io/2019/03/performance-tips-of-numpy-ndarray.html)

1. [Beating NumPy performance speed by extending Python with C](https://medium.com/analytics-vidhya/beating-numpy-performance-by-extending-python-with-c-c9b644ee2ca8)

1. [Principles of Performance](https://llllllllll.github.io/principles-of-performance/index.html)